In [2]:
# Remember to install JDK 
#   -  https://www.oracle.com/java/technologies/downloads/
# Remember to install pyspark and findspark
#   - pip install findspark
#   - pip install pyspark

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
    .appName("HelloWorld") \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('WARN')

24/05/20 21:26:56 WARN Utils: Your hostname, Rauls-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.8 instead (on interface en0)
24/05/20 21:26:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/20 21:26:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
nums = sc.parallelize([1,2,3,4])
print(nums.map(lambda x: x*x).collect())

[1, 4, 9, 16]


In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

spark = SparkSession.builder \
    .appName("BibleWordCount") \
    .getOrCreate()

bible_text = spark.read.text("/Users/raulperbichi/Desktop/School/Spring 2024/CS 452/CS452/bible-1.txt")

words_df = bible_text.select(explode(split(bible_text.value, "\\s+")).alias("word"))

filtered_words_df = words_df.filter(words_df.word != "")

word_counts_df = filtered_words_df.groupBy("word").count()

word_counts_df.createOrReplaceTempView("word_counts")

top_words = spark.sql("""
SELECT word, count 
FROM word_counts 
ORDER BY count DESC
LIMIT 20
""")

top_words.show()

spark.stop()


+-----+-----+
| word|count|
+-----+-----+
|  the|62257|
|  and|38642|
|   of|34553|
|   to|13457|
|  And|12735|
| that|12464|
|   in|12222|
|shall| 9762|
|   he| 9510|
| unto| 8932|
|    I| 8708|
|  his| 8362|
|    a| 7999|
|  for| 7161|
| they| 6895|
|   be| 6736|
|   is| 6721|
| with| 5997|
|  not| 5859|
|  all| 5256|
+-----+-----+

